<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=2fcfb46e2b65f867929141bbb6ff17e575e94b58b697c8ae99ab674545f2b0c6
  Stored in directory: /home/runner/.cache/pip/wheels/ec/b6/81/d781ff040ee9432384560f29596ee14ca79d70529d4228e50f
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.2
    Uninstalling protobuf-6.33.2:
      Successfully uninstalled protobuf-6.33.2


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.66
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-12-18 13:46:28
-------------------
qualified stocks: 86
with latest results: 25
still star stocks: 15
-------------------
Initial Investment:  1.28 C
CY Investment:  1.55 C
Reserve:  66.76 K
Current:  1.39 C
-------------------
Today PnL: -12.95 K (-0.09%)
Current PnL: -27.89 L (-18.02%)
CY Booked + Current PnL: -13.60 L (-8.79%)
-------------------
Total profit:  1.39 L
Total loss:  -29.28 L
-------------------
Total Booked + Current PnL: 13.38 L (10.47%)
Total Booked PnL: 41.27 L (32.29%)
Curr Year Booked PnL: 14.29 L (10.27%)
Prev Year Booked PnL: 26.98 L (21.11%)
Est FTT:  2.32 C
Est FTT PnL: 92.53 L (66.49%)
Deployed:  1.28 C
Current:  1.39 C
CAGR/XIRR %: 4.71%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT','Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'FTT Amt', 'Current P/L', 'Current', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%',  'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
82,VOLTAS,1530.00,2.14,10.31,8.50,19.69,17979.0,19773.0,211515.0,-0.39,61.0,X-MC,2.53,78.0,1.10,1.53,17.81,XY25,NTT,AC
77,TTKPRESTIG,770.00,-1.29,-20.21,25.45,0.09,20464.0,-20369.0,80408.0,86.55,27.0,M-SC,2.38,253.0,-1.00,0.58,5.64,OX40N,NTT,DURABLES
50,MASFIN,398.61,-0.83,-4.33,27.58,22.05,25852.0,-4245.0,93735.0,-17.34,48.0,H-SC,6.76,164.0,-0.16,0.68,36.25,XR,ATH,FINANCE
39,INDIGOPNTS,1408.00,-1.65,-16.29,19.48,0.02,28465.0,-28437.0,146122.0,122.66,41.0,M-SC,13.66,234.0,-1.00,1.06,27.10,OX40N,NTT,PAINTS
43,ITC,452.00,0.24,-1.78,12.79,10.78,30190.0,-4276.0,236042.0,-41.56,44.0,X-LC,2.06,5.0,-0.14,1.70,3.36,X40,NTT,FMCG


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
61,ROUTE,2227.21,12.42,-41.94,191.14,69.05,147684.0,-55801.0,77265.0,-53.08,64.0,H-SC,22.13,140.0,-0.38,0.56,19.16,SR,ATH,IT
6,ASIANTILES,137.00,5.25,-10.13,102.15,81.67,86119.0,-9504.0,84306.0,7583.33,62.0,L-SC,12.47,271.0,-0.11,0.61,63.26,XR,NTT,CERAMICS
82,VOLTAS,1530.00,2.14,10.31,8.50,19.69,17979.0,19773.0,211515.0,-0.39,61.0,X-MC,2.53,78.0,1.10,1.53,17.81,XY25,NTT,AC
71,TANLA,1943.92,2.02,-36.41,245.49,119.69,422663.0,-98590.0,172171.0,-35.93,46.0,H-SC,7.82,156.0,-0.23,1.24,36.01,AR,ATH,IT
15,CAMS,950.00,1.76,-2.16,27.23,24.48,67098.0,-5445.0,246411.0,-81.32,43.0,X-SC,0.58,86.0,-0.08,1.78,20.48,X40N,NTT,MISC


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
56,RAJESHEXPO,518.0,-3.39,-63.48,174.00,0.07,87171.0,-87079.0,50098.0,1735.86,50.0,L-SC,2.35,268.0,-1.00,0.36,24.60,OX40N,NTT,JEWELLERY
80,VALIANTORG,838.0,-2.82,-46.22,223.80,74.15,196926.0,-75613.0,87992.0,-258.45,39.0,H-SC,22.87,149.0,-0.38,0.64,12.79,XR,NTT,CHEMICALS
16,CERA,9475.0,-2.50,-30.77,78.94,23.89,96137.0,-54118.0,121785.0,-33.17,39.0,H-SC,11.06,157.0,-0.56,0.88,5.18,OX40N,NTT,CERAMICS
65,SIEMENS,4671.5,-2.28,-17.55,52.24,25.51,80152.0,-32665.0,153430.0,-5.83,38.0,H-LC,3.42,49.0,-0.41,1.11,13.07,AR,ATH,ELECTRICAL
14,CAMPUS,393.0,-2.24,-13.25,53.64,33.28,78073.0,-22225.0,145550.0,-30.08,33.0,M-SC,2.28,222.0,-0.28,1.05,12.81,XY24,NTT,FOOTWEAR


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
25,FINCABLES,1641.55,0.97,-2.01,112.18,107.91,162294.0,-2971.0,144673.0,-20.15,56.0,M-SC,12.54,216.0,-0.02,1.04,7.87,OX40N,ATH,CABLES
39,INDIGOPNTS,1408.00,-1.65,-16.29,19.48,0.02,28465.0,-28437.0,146122.0,122.66,41.0,M-SC,13.66,234.0,-1.00,1.06,27.10,OX40N,NTT,PAINTS
77,TTKPRESTIG,770.00,-1.29,-20.21,25.45,0.09,20464.0,-20369.0,80408.0,86.55,27.0,M-SC,2.38,253.0,-1.00,0.58,5.64,OX40N,NTT,DURABLES
66,SIS,528.00,-0.05,-24.25,60.07,21.26,50526.0,-26920.0,84112.0,1984.46,50.0,H-SC,4.96,166.0,-0.53,0.61,13.64,OX40N,NTT,MISC
47,KANSAINER,340.00,-0.14,-25.99,53.31,13.47,106403.0,-70074.0,199593.0,-69.48,32.0,H-SC,6.05,158.0,-0.66,1.44,1.49,XY24,NTT,PAINTS


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
84,WIPRO,420.0,0.48,7.62,60.06,72.27,109530.0,12920.0,182368.0,-7.44,67.0,M-LC,2.79,99.0,0.12,1.32,14.27,XR,NTT,IT


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
84,WIPRO,420.00,0.48,7.62,60.06,72.27,109530.0,12920.0,182368.0,-7.44,67.0,M-LC,2.79,99.0,0.12,1.32,14.27,XR,NTT,IT
25,FINCABLES,1641.55,0.97,-2.01,112.18,107.91,162294.0,-2971.0,144673.0,-20.15,56.0,M-SC,12.54,216.0,-0.02,1.04,7.87,OX40N,ATH,CABLES
38,INDIAMART,4810.62,1.35,-2.58,112.19,106.72,134803.0,-3180.0,120156.0,-51.77,42.0,H-SC,3.40,139.0,-0.02,0.87,20.20,AR,ATH,MISC
85,ZYDUSLIFE,1286.17,-0.48,-3.65,40.77,35.63,82322.0,-7657.0,201917.0,-17.18,36.0,H-MC,2.77,119.0,-0.09,1.46,12.41,AR,ATH,PHARMA
37,IEX,219.00,0.21,-4.64,55.85,48.63,106498.0,-9268.0,190686.0,-36.48,47.0,H-SC,13.54,136.0,-0.09,1.38,6.38,XR,NTT,MISC


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
9,BAJAJHFL,181.50,0.01,-18.97,91.86,55.47,181098.0,-46140.0,197146.0,-28.25,23.0,X-MC,13.94,64.0,-0.25,1.42,1.01,X40N,ATH,FINANCE
53,PAGEIND,51605.07,-0.76,-11.18,44.23,28.11,79127.0,-22510.0,178900.0,-35.61,24.0,X-MC,13.47,55.0,-0.28,1.29,0.00,X40,ATH,APPARELS
8,AWL,485.00,-1.37,-24.63,103.65,53.50,235734.0,-74318.0,227433.0,-63.50,27.0,X-MC,3.80,58.0,-0.32,1.64,1.02,XY24,NTT,FMCG
11,BATAINDIA,2096.00,0.02,-39.40,123.10,35.20,95991.0,-50692.0,77978.0,2.24,29.0,X-SC,16.45,92.0,-0.53,0.56,0.02,X40,NTT,FOOTWEAR
33,HINDUNILVR,2922.00,-0.47,-11.89,29.01,13.67,65708.0,-30563.0,226500.0,-18.45,30.0,X-LC,2.47,14.0,-0.47,1.64,6.93,XY25,NTT,FMCG


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
15,CAMS,950.0,1.76,-2.16,27.23,24.48,67098.0,-5445.0,246411.0,-81.32,43.0,X-SC,0.58,86.0,-0.08,1.78,20.48,X40N,NTT,MISC
12,BERGEPAINT,680.0,-1.09,-5.05,27.58,21.14,59536.0,-11474.0,215865.0,-11.79,36.0,X-MC,1.69,74.0,-0.19,1.56,21.67,XY24,NTT,PAINTS
20,DABUR,735.0,-0.33,-3.71,49.24,43.70,117616.0,-9206.0,238862.0,-11.62,34.0,X-MC,1.98,72.0,-0.08,1.72,10.78,XY24,BTT,FMCG
43,ITC,452.0,0.24,-1.78,12.79,10.78,30190.0,-4276.0,236042.0,-41.56,44.0,X-LC,2.06,5.0,-0.14,1.70,3.36,X40,NTT,FMCG
78,UNITDSPR,1644.0,-2.16,2.41,17.93,20.77,42993.0,5641.0,239785.0,-6.55,40.0,X-MC,2.35,62.0,0.13,1.73,9.49,X40N,NTT,BREWERIES


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
3,ACC,3906.00,-0.19,-26.22,122.50,64.15,215049.0,-62401.0,175550.0,-56.75,31.0,X-SC,3.01,82.0,-0.29,1.27,0.00,XY24,BTT,CEMENT
19,COLPAL,3726.84,-0.16,-20.87,78.83,41.51,164282.0,-54965.0,208400.0,-5.86,34.0,X-MC,7.52,61.0,-0.33,1.50,0.00,XY25,ATH,FMCG
53,PAGEIND,51605.07,-0.76,-11.18,44.23,28.11,79127.0,-22510.0,178900.0,-35.61,24.0,X-MC,13.47,55.0,-0.28,1.29,0.00,X40,ATH,APPARELS
11,BATAINDIA,2096.00,0.02,-39.40,123.10,35.20,95991.0,-50692.0,77978.0,2.24,29.0,X-SC,16.45,92.0,-0.53,0.56,0.02,X40,NTT,FOOTWEAR
59,RELAXO,1176.00,-0.47,-48.28,194.48,52.31,146012.0,-70082.0,75078.0,-44.92,39.0,X-SC,7.26,91.0,-0.48,0.54,0.50,X40N,NTT,FOOTWEAR


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
55,QUESS,424.00,-0.41,-31.21,107.66,42.84,48140.0,-20291.0,44715.0,-54.82,38.0,X-SC,35.18,83.0,-0.42,0.32,1.16,XY24,NTT,MISC
59,RELAXO,1176.00,-0.47,-48.28,194.48,52.31,146012.0,-70082.0,75078.0,-44.92,39.0,X-SC,7.26,91.0,-0.48,0.54,0.50,X40N,NTT,FOOTWEAR
11,BATAINDIA,2096.00,0.02,-39.40,123.10,35.20,95991.0,-50692.0,77978.0,2.24,29.0,X-SC,16.45,92.0,-0.53,0.56,0.02,X40,NTT,FOOTWEAR
51,MEDANTA,1486.00,-0.97,-5.51,31.32,24.08,37568.0,-7000.0,119950.0,-13.19,31.0,X-SC,5.39,89.0,-0.19,0.87,13.02,XY24,NTT,HEALTHCARE
34,HONAUT,58357.33,-0.77,-17.53,73.79,43.32,99115.0,-28552.0,134320.0,-29.17,33.0,X-SC,5.03,90.0,-0.29,0.97,3.13,X40N,ATH,ELECTRICAL


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
73,TCS,4389.97,1.69,-10.03,34.16,20.71,116250.0,-37928.0,340309.0,-22.65,68.0,X-LC,4.05,1.0,-0.33,2.46,13.71,X40,ATH,IT
41,INFY,1972.00,1.30,10.47,21.52,34.24,74385.0,32768.0,345656.0,-13.85,66.0,X-LC,4.56,2.0,0.44,2.50,19.62,X40,NTT,IT
75,TMPV,600.00,-0.04,-29.04,73.31,22.98,117509.0,-65607.0,160291.0,-26.74,31.0,X-LC,6.68,3.0,-0.56,1.16,0.82,XY24,NTT,AUTO
81,VBL,671.64,-0.13,-5.01,42.83,35.67,128499.0,-15822.0,300020.0,-16.24,49.0,X-LC,4.05,4.0,-0.12,2.17,8.15,X40N,ATH,FMCG
43,ITC,452.00,0.24,-1.78,12.79,10.78,30190.0,-4276.0,236042.0,-41.56,44.0,X-LC,2.06,5.0,-0.14,1.70,3.36,X40,NTT,FMCG


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
6,ASIANTILES,137.00,5.25,-10.13,102.15,81.67,86119.0,-9504.0,84306.0,7583.33,62.0,L-SC,12.47,271.0,-0.11,0.61,63.26,XR,NTT,CERAMICS
50,MASFIN,398.61,-0.83,-4.33,27.58,22.05,25852.0,-4245.0,93735.0,-17.34,48.0,H-SC,6.76,164.0,-0.16,0.68,36.25,XR,ATH,FINANCE
13,BSOFT,831.70,-0.15,-21.82,92.88,50.79,102127.0,-30697.0,109956.0,0.12,64.0,H-SC,6.59,151.0,-0.30,0.79,28.02,XR,ATH,IT
39,INDIGOPNTS,1408.00,-1.65,-16.29,19.48,0.02,28465.0,-28437.0,146122.0,122.66,41.0,M-SC,13.66,234.0,-1.00,1.06,27.10,OX40N,NTT,PAINTS
5,ASIANPAINT,3460.25,-0.85,-3.46,25.28,20.95,61447.0,-8703.0,243065.0,-4.43,45.0,X-LC,16.83,31.0,-0.14,1.76,30.95,X40,ATH,PAINTS


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
13,BSOFT,831.70,-0.15,-21.82,92.88,50.79,102127.0,-30697.0,109956.0,0.12,64.0,H-SC,6.59,151.0,-0.30,0.79,28.02,XR,ATH,IT
61,ROUTE,2227.21,12.42,-41.94,191.14,69.05,147684.0,-55801.0,77265.0,-53.08,64.0,H-SC,22.13,140.0,-0.38,0.56,19.16,SR,ATH,IT
41,INFY,1972.00,1.30,10.47,21.52,34.24,74385.0,32768.0,345656.0,-13.85,66.0,X-LC,4.56,2.0,0.44,2.50,19.62,X40,NTT,IT
84,WIPRO,420.00,0.48,7.62,60.06,72.27,109530.0,12920.0,182368.0,-7.44,67.0,M-LC,2.79,99.0,0.12,1.32,14.27,XR,NTT,IT
73,TCS,4389.97,1.69,-10.03,34.16,20.71,116250.0,-37928.0,340309.0,-22.65,68.0,X-LC,4.05,1.0,-0.33,2.46,13.71,X40,ATH,IT


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,20.41
1,25,44.89
2,50,76.73


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    44.80
MC    30.94
LC    24.27
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     26.56
X40      23.21
X40N     12.99
XR        9.79
XY25      9.14
AR        8.99
OX40N     7.52
SR        1.05
MH        0.76
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
X-MC    24.47
H-SC    23.76
X-LC    20.81
M-SC    11.57
X-SC     8.04
H-MC     4.79
L-SC     1.43
M-MC     1.34
M-LC     1.32
H-LC     1.11
L-LC     1.03
L-MC     0.34
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,17.33,-8.72,43.68
IT,13.24,-14.66,72.31
FINANCE,9.44,-21.88,74.89
MISC,7.29,-28.60,81.20
ELECTRICAL,5.98,-13.35,54.87
PAINTS,5.82,-14.75,31.80
INSURANCE,4.82,-1.69,36.49
PHARMA,4.15,-2.35,34.57
AUTO,2.81,-34.56,81.21


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3241773.0,21
AR,1324910.0,10
XR,1307811.0,13
X40,1011150.0,14
X40N,886961.0,9
OX40N,750031.0,10
XY25,375949.0,6
SR,275816.0,2
MH,78337.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3688609.0,25
M-SC,1454842.0,15
X-MC,1437648.0,16
X-LC,846684.0,11
X-SC,806072.0,8
H-MC,414907.0,3
L-SC,262918.0,3
M-LC,109530.0,1
H-LC,80152.0,1


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1297909.0      6
           AR         899860.0      5
M-SC       XY24       834171.0      6
H-SC       XR         791016.0      7
X-MC       X40        494324.0      7
           XY24       412886.0      3
           X40N       348177.0      4
H-SC       OX40N      345671.0      4
X-LC       X40        323736.0      5
M-SC       OX40N      317189.0      5
X-SC       X40N       312225.0      3
           XY24       300757.0      3
H-SC       SR         275816.0      2
X-LC       X40N       226559.0      2
H-MC       AR         217947.0      2
X-LC       XY24       199090.0      2
H-MC       XY24       196960.0      1
X-SC       X40        193090.0      2
X-MC       XY25       182261.0      2
M-SC       XR         176531.0      2
L-SC       XR         175747.0      2
M-SC       AR         126951.0      2
M-LC       XR         109530.0      1
X-LC       XY25        97299.0      2
L-SC       OX40N       87171.0      1
H-LC       AR          80152.0      1
H-SC       MH          78337.0      1
M-MC       XY25        59565.0      1
L-MC       XR          54987.0      1
L-LC       XY25        36824.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 29.0 seconds
